In [1]:
import pandas as pd

In [2]:
likes = pd.read_csv('C:/Users/maslo/курсы/karpov_start_ml/4_module/11_lesson/materials/likes.csv')

In [3]:
likes

,user_id,post_id,timestamp
0,128381,4704,1654030804
1,146885,1399,1654030816
2,50948,2315,1654030828
3,14661,673,1654030831
4,37703,1588,1654030833
...,...,...,...
230171,31851,5964,1655243535
230172,51512,1498,1655243537
230173,34017,5009,1655243573
230174,13267,1787,1655243692


In [64]:
len(likes)

230176

In [4]:
views = pd.read_csv('C:/Users/maslo/курсы/karpov_start_ml/4_module/11_lesson/materials/views.csv')

In [5]:
views

,user_id,exp_group,recommendations,timestamp
0,128381,control,[3644 4529 4704 5294 4808],1654030803
1,146885,test,[1399 1076 797 7015 5942],1654030811
2,50948,test,[2315 3037 1861 6567 4093],1654030825
3,37703,test,[2842 1949 162 1588 6794],1654030826
4,14661,test,[2395 5881 5648 3417 673],1654030829
...,...,...,...,...
193290,158267,test,[1733 6834 4380 1915 1627],1655240340
193291,63527,control,[2454 191 3873 6404 1588],1655240347
193292,52169,test,[1368 1709 1616 798 5305],1655240354
193293,142402,test,[5895 6984 1978 6548 6106],1655240373


In [65]:
len(views)

193295

In [15]:
views\
.groupby('user_id')['exp_group']\
.nunique()\
.reset_index()\
.rename(columns={'exp_group':'uq_exp_group'})\
.sort_values(by='uq_exp_group', ascending=False)\
.head(10)

# 142283, 148670, 55788, 25623

,user_id,uq_exp_group
54475,142283,2
57065,148670,2
20633,55788,2
10071,25623,2
43341,114278,1
43345,114289,1
43344,114288,1
43343,114284,1
43342,114279,1
43339,114273,1


In [16]:
bug_users = [142283, 148670, 55788, 25623]

In [18]:
views[~views['user_id'].isin(bug_users)].groupby('exp_group')['user_id'].nunique()

exp_group
control    32350
test       32659
Name: user_id, dtype: int64

In [21]:
recommendation_users = list(views['user_id'].unique())

In [24]:
liked_recommendation_users = likes[likes['user_id'].isin(recommendation_users)]['user_id'].unique()

In [26]:
len(liked_recommendation_users) / len(recommendation_users)

0.8947595096365343

In [27]:
len(likes[likes['user_id'].isin(recommendation_users)]) / len(likes)

1.0

In [39]:
user_m_group = views[~views['user_id'].isin(bug_users)][['user_id', 'exp_group']]

In [41]:
user_m_group.drop_duplicates(keep='first', inplace=True)

In [42]:
user_m_group

,user_id,exp_group
0,128381,control
1,146885,test
2,50948,test
3,37703,test
4,14661,test
...,...,...
193267,80500,test
193270,149686,test
193285,3615,control
193288,119630,test


In [43]:
len(views), len(user_m_group)

(193295, 65009)

In [45]:
likes.columns

Index(['user_id', 'post_id', 'timestamp'], dtype='object')

In [49]:
likes[~likes['user_id'].isin(bug_users)]\
.merge(user_m_group, 
       how='left', 
       left_on='user_id',
       right_on='user_id')\
.groupby('exp_group')['user_id'].nunique().reset_index()

,exp_group,user_id
0,control,28834
1,test,29333


In [51]:
views[~views['user_id'].isin(bug_users)].groupby('exp_group')['user_id'].nunique().reset_index()

,exp_group,user_id
0,control,32350
1,test,32659


In [68]:
control_at_least_1_like_rate = 28834/32350

In [69]:
print(f'control at lest 1 likes rate = {control_at_least_1_like_rate}')

control at lest 1 likes rate = 0.8913137557959815


In [70]:
test_at_least_1_like_rate = 29333/32659

In [71]:
print(f'test at lest 1 likes rate = {test_at_least_1_like_rate}')

test at lest 1 likes rate = 0.8981597721914327


In [63]:
likes[~likes['user_id'].isin(bug_users)]\
.merge(user_m_group, 
       how='left', 
       left_on='user_id',
       right_on='user_id')\
.groupby('exp_group')['user_id'].count().reset_index()

,exp_group,user_id
0,control,112807
1,test,117330


In [74]:
control_likes_per_user = 112807/32350

In [75]:
print(f'control likes per user = {control_likes_per_user}')

control likes per user = 3.487078825347759


In [76]:
test_likes_per_user = 117330/32659

In [77]:
print(f'test likes per user = {test_likes_per_user}')

test likes per user = 3.592577849903549


In [81]:
likes_in_groups = \
likes[~likes['user_id'].isin(bug_users)]\
.merge(user_m_group, 
       how='left', 
       left_on='user_id',
       right_on='user_id').copy()

In [86]:
views.columns

Index(['user_id', 'exp_group', 'recommendations', 'timestamp'], dtype='object')

In [82]:
len(likes_in_groups)

230137

In [83]:
likes_in_groups.columns

Index(['user_id', 'post_id', 'timestamp', 'exp_group'], dtype='object')

In [91]:
from numpy import random

In [101]:
views_search = views[~views['user_id'].isin(bug_users)]

control_at_least_1_like_rates = []
test_at_least_1_like_rates = []

control_likes_per_user = []
test_likes_per_user = []

control_user_ids = list(views_search[views_search['exp_group'] == 'control']['user_id'].unique())
test_user_ids = list(views_search[views_search['exp_group'] == 'test']['user_id'].unique())

for i in range(1000):
    
    control_bag_user_ids = random.choice(control_user_ids, len(control_user_ids))
    test_bag_user_ids = random.choice(test_user_ids, len(test_user_ids))
    
    control_at_least_1_like_rates.append(
        likes_in_groups[likes_in_groups['user_id'].isin(control_bag_user_ids)]['user_id'].nunique() / len(control_bag_user_ids)
    )
    test_at_least_1_like_rates.append(
        likes_in_groups[likes_in_groups['user_id'].isin(test_bag_user_ids)]['user_id'].nunique() / len(test_bag_user_ids)
    )
    control_likes_per_user.append(
        len(likes_in_groups[likes_in_groups['user_id'].isin(control_bag_user_ids)]) / len(control_bag_user_ids)
    )
    test_likes_per_user.append(
        len(likes_in_groups[likes_in_groups['user_id'].isin(test_bag_user_ids)]) / len(test_bag_user_ids)
    )

In [110]:
import scipy.stats as sps

# Ваши два числа
x = control_at_least_1_like_rates
y = test_at_least_1_like_rates

# Применение критерия Манна-Уитни
u, p_value = sps.mannwhitneyu(x, y, alternative='two-sided')

In [111]:
p_value

4.071518509918308e-276

In [112]:
x = control_likes_per_user
y = test_likes_per_user

# Применение критерия Манна-Уитни
u, p_value = sps.mannwhitneyu(x, y, alternative='two-sided')

In [113]:
p_value

0.0

In [114]:
likes

,user_id,post_id,timestamp
0,128381,4704,1654030804
1,146885,1399,1654030816
2,50948,2315,1654030828
3,14661,673,1654030831
4,37703,1588,1654030833
...,...,...,...
230171,31851,5964,1655243535
230172,51512,1498,1655243537
230173,34017,5009,1655243573
230174,13267,1787,1655243692


In [126]:
likes.dtypes

user_id      int64
post_id      int64
timestamp    int64
dtype: object

In [115]:
views

,user_id,exp_group,recommendations,timestamp
0,128381,control,[3644 4529 4704 5294 4808],1654030803
1,146885,test,[1399 1076 797 7015 5942],1654030811
2,50948,test,[2315 3037 1861 6567 4093],1654030825
3,37703,test,[2842 1949 162 1588 6794],1654030826
4,14661,test,[2395 5881 5648 3417 673],1654030829
...,...,...,...,...
193290,158267,test,[1733 6834 4380 1915 1627],1655240340
193291,63527,control,[2454 191 3873 6404 1588],1655240347
193292,52169,test,[1368 1709 1616 798 5305],1655240354
193293,142402,test,[5895 6984 1978 6548 6106],1655240373


In [127]:
views.dtypes

user_id             int64
exp_group          object
recommendations    object
timestamp           int64
dtype: object

In [117]:
! pip install pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-cp37-none-any.whl size=26823 sha256=cb431b81ba71297f7891e29fb4eba405e5b864a7855393817444d9b688030ace
  Stored in directory: C:\Users\maslo\AppData\Local\pip\Cache\wheels\53\6c\18\b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [118]:
! pip install SQLAlchemy==1.4.17

  Found existing installation: SQLAlchemy 1.3.9
    Uninstalling SQLAlchemy-1.3.9:
      Successfully uninstalled SQLAlchemy-1.3.9


In [119]:
from pandasql import sqldf

In [133]:
q = \
'''
select count(1) as cnt from (
    select v.user_id, v.timestamp
      from views v 
           left join likes l on v.timestamp <= l.timestamp 
                                and l.timestamp - v.timestamp <= 3600
                                and (v.recommendations like '[' || l.post_id || ' %' or 
                                     v.recommendations like '% ' || l.post_id || ' %' or 
                                     v.recommendations like '% ' || l.post_id || ']' 
                                     )
                                and l.user_id = v.user_id
     group by v.user_id, v.timestamp
    having sum(case when l.timestamp is not null then 1 else 0 end)
) t 
'''

In [134]:
pd.DataFrame(sqldf(q, locals()))

,cnt
0,137871


In [135]:
137871/193295 

0.7132672857549341

In [150]:
q = \
'''
select t.exp_group, t.user_id % 100 bucket_id, 
       cast(sum(case when user_rec_like > 0 then 1 else 0 end) as real) / count(1) hitrate
from (
    select v.exp_group, v.user_id, v.timestamp, 
           sum(case when l.timestamp is not null then 1 else 0 end) user_rec_like
      from views v 
           left join likes l on v.timestamp <= l.timestamp 
                                and l.timestamp - v.timestamp <= 3600
                                and (v.recommendations like '[' || l.post_id || ' %' or 
                                     v.recommendations like '% ' || l.post_id || ' %' or 
                                     v.recommendations like '% ' || l.post_id || ']' 
                                     )
                                and l.user_id = v.user_id
     group by v.exp_group, v.user_id, v.timestamp
) t 
group by t.exp_group, t.user_id % 100
'''

In [151]:
group_bucket_hitrate = pd.DataFrame(sqldf(q, locals()))

In [152]:
group_bucket_hitrate

,exp_group,bucket_id,hitrate
0,control,0,0.754527
1,control,1,0.673077
2,control,2,0.698152
3,control,3,0.724563
4,control,4,0.716683
...,...,...,...
195,test,95,0.727088
196,test,96,0.715363
197,test,97,0.722004
198,test,98,0.744533


In [155]:
x = group_bucket_hitrate[group_bucket_hitrate['exp_group'] == 'control']['hitrate']
y = group_bucket_hitrate[group_bucket_hitrate['exp_group'] == 'test']['hitrate']

# Применение критерия Манна-Уитни
u, p_value = sps.mannwhitneyu(x, y, alternative='two-sided')

In [156]:
p_value

1.514033009870076e-07

In [153]:
q = \
'''
select t.exp_group,
       cast(sum(case when user_rec_like > 0 then 1 else 0 end) as real) / count(1) hitrate
from (
    select v.exp_group, v.user_id, v.timestamp, 
           sum(case when l.timestamp is not null then 1 else 0 end) user_rec_like
      from views v 
           left join likes l on v.timestamp <= l.timestamp 
                                and l.timestamp - v.timestamp <= 3600
                                and (v.recommendations like '[' || l.post_id || ' %' or 
                                     v.recommendations like '% ' || l.post_id || ' %' or 
                                     v.recommendations like '% ' || l.post_id || ']' 
                                     )
                                and l.user_id = v.user_id
     group by v.exp_group, v.user_id, v.timestamp
) t 
group by t.exp_group
'''

In [154]:
pd.DataFrame(sqldf(q, locals()))

,exp_group,hitrate
0,control,0.706655
1,test,0.719843
